In [1]:
import pandas as pd
pd.options.display.max_rows = 200
import numpy as np 
# pd.set_option('display.max_colwidth', None)
import os

In [ ]:
# now zipped in Summarization_and_preprocessig_dfs.zip and Summarization_and_preprocessig_dfs_2.zip

In [ ]:
df['generated_transcripts'].str.len()

In [ ]:
gen_str_lens = df['generated_transcripts'].str.len().to_list()

In [ ]:
np.average(gen_str_lens).round(2)

In [ ]:
df['generated_transcripts'].iloc[:1].

In [ ]:
df.info()

# **PREPROCESSING BEFORE SUMMARIZATION**

In [2]:
import re
from spellchecker import SpellChecker
import spacy
from contractions import contractions_dict
from textblob import TextBlob
from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification

# Spell checker
spell = SpellChecker()

# Loadng English tokenizer, tagger, parser, NER, and word vectors
nlp = spacy.load("en_core_web_sm")

# **Text Cleaning and Processing Function**

In [3]:
contraction_mapping = contractions_dict


def lower_dict(d):
    return {k.lower(): v.lower() for k, v in d.items()}

contraction_mapping = lower_dict(contractions_dict)

In [4]:
def expand_contractions(text, contraction_mapping):
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())),
                                      flags=re.IGNORECASE | re.DOTALL)

    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match, 
                        contraction_mapping.get(match.lower(), match)) # Default to match if not found
        # Ensure the first character's case is preserved
        expanded_contraction = first_char + expanded_contraction[1:] if expanded_contraction else match
        return expanded_contraction

    expanded_text = contractions_pattern.sub(expand_match, text)
    return expanded_text

spell_check_cache = {}

def correct_spellings_cached(text):
    words = text.split()
    corrected_words = []
    for word in words:
        if word not in spell_check_cache:
            corrected_word = spell.correction(word) if spell.correction(word) else word
            spell_check_cache[word] = corrected_word
        else:
            corrected_word = spell_check_cache[word]
        corrected_words.append(corrected_word)
    return ' '.join(corrected_words)

In [11]:
def to_sentence_case(text):
    """Converts text to sentence case."""
    # Pattern to match sentence endings. Adjust as needed for abbreviations, etc.
    sentence_endings = re.compile('(?<=[.!?]) +')
    sentences = sentence_endings.split(text)
    return ' '.join(sentence.capitalize() for sentence in sentences)

def preprocess_text_NEW(text, contraction_mapping):
    entity_placeholders = {}
    doc = nlp(text)
    processed_text = text
    for ent in reversed(doc.ents):
        placeholder = f"<ENTITY_{ent.label_}_{ent.start_char}>".lower()  # Lowercase the placeholder
        entity_placeholders[placeholder] = ent.text
        processed_text = processed_text[:ent.start_char] + placeholder + processed_text[ent.end_char:]
    
    processed_text = processed_text.lower()
    processed_text = expand_contractions(processed_text, contraction_mapping)
    processed_text = re.sub(r'\b(?:' + '|'.join(['um', 'uh', 'you know', 'so', 'like']) + r')\b', '', processed_text)
    processed_text = re.sub(r'\s+', ' ', processed_text)
    processed_text = re.sub(r'\s([?.!",;](?:\s|$))', r'\1', processed_text)
    processed_text = correct_spellings_cached(processed_text) # Uncomment to enable spell checking
    processed_text = to_sentence_case(processed_text)
    
    for placeholder, original_text in entity_placeholders.items():
        processed_text = processed_text.replace(placeholder, original_text.upper())
    
    return processed_text


In [12]:
x = "Dr. John Smith's groundbroking research on CRISPR gene editing at Harvard University wasn't that interesting"
y = "FROM W B EASY CHICAGO IT'S THIS AMERICAN LIFE I'M NANCY UPDIKE FILLING IN FOR IRA GLASS TI DA SHOW IS A RERUN A GOOD ON AND I'M GIN A START WITH THIS STORY THAT I WANT A SHARE IT'S LITTLE PERSONAL I WAS AT MAC THE MAKE UP STORE NOT THE COMPUTERS STORE AND I WAS BUYING FOUNDATION WHICH I ALMOST NEVER WEAR IT'S THE MAKE UP YOU PUT ALL OVER YOUR FACE TO GIVE YOURSELF THE PRETEND PERFECT SKIN AND I ASKED THE SALESMAN FOR HELP FINDING THE RIGHT COLOR AND HE LOOKED AT ME AND HE SAID ALMOST LIKE YOU WAS THINKING OUT LOUD HE SAID YOUR NECK IT'S SO MUCH MORE YELLOW THAN YOUR FACE AND THEN HE TURNED AWAY TO START LOOKING FOR THE IMPOSSIBLE COLOR THAT WOULD SOLVE THIS PROBLEM OF THE YELLOW WRIHT NEXT TO THE SO MUCH MORE YELLOW AND IF YOU'RE THINKING ALL THIS WAS JUST A SALE'S TECHNIQUE TO INVENT A PROBLEM AND THEN OFFERD TO FIX IT WITH MORE PRODUCTS I WISH THAT THAT HAD BEEN THE CASE BUT THIS WAS NOT AN UPSELL THIS WAS A CREE TO CUR THE MAN REALLY JUST SEEMED TO BE EXPRESSING HIS FRUSTRATION AT THIS STUMPER OF MY MISMATCHED FACE AND NECK"
print(preprocess_text_NEW(x, contraction_mapping))
print(preprocess_text_NEW(y, contraction_mapping))
print('\n')


# reconstructed_text = reconstruct_text_with_punctuation(y, pipe(y.lower()))

# print(reconstructed_text)




Dry JOHN SMITH's groundbreaking research on CRISPR gene editing at HARVARD UNIVERSITY was not that interesting
From w b easy chico it is this america life i am nancy unlike filling in for era glass ti a show is a rerun a good on and i am gin a start with this story that i want a share it is little personal i was at MAC the make up store not the computers store and i was buying foundation which i almost never wear it is the make up you put all over your face to give yourself the pretend perfect skin and i asked the salesman for help finding the right color and he looked at me and he said almost you was thinking out loud he said your neck it is much more yellow than your face and then he turned away to start looking for the impossible color that would solve this problem of the yellow wrist next to the much more yellow and if you are thinking all this was just a sale's technique to invent a PROBLEM and then offer to fix it with more products i wish that that had been the case but this was

In [ ]:
# df['cleaned_generated_text'] = df['generated_trans'].apply(lambda x: preprocess_text_NEW(x, contraction_mapping))
df['cleaned_generated_text'] = df['generated_transcripts'].apply(lambda x: preprocess_text_NEW(x, contraction_mapping))


In [13]:
sample_file = 'BEN_GENERATED_TRANS/39.txt'

with open(sample_file, 'r') as file:
    sample = file.read()

with open('sample_preprocessed_text_6.txt_with_NER_and_Spell_Check', 'w') as out_file: 
    out_file.write(preprocess_text_NEW(sample, contraction_mapping))


In [ ]:
df.to_csv('s_and_h_preprocess_1.csv')

In [ ]:
df